In [1]:
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd

In [11]:
def sigmond(x):
    return 1/(1+np.exp(-np.abs(x)))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

In [12]:
###### 读取数据 
stock_data = pd.read_csv('stock_data_normal.csv',header=None)
stock_lable = pd.read_csv('stock_data_lable_normal.txt',header=None)

In [13]:
#print(stock_lable.head(10))

In [14]:
lab_count = stock_lable.shape[0]
x = stock_data.loc[0:59999].values
print(x.shape)
x = x.reshape(-1,66)
print(x.shape,x[0:3][0:3])
y = stock_lable.loc[0:9999,2].values
y = y.reshape(-1,1)
print(y.shape,y[0:10])
x = x / 2
x = np.insert(x,0,1,axis=1)

(60000, 11)
(10000, 66) [[-1.79289e+00 -6.80910e-01 -7.93950e-01 -1.65258e+00  1.61912e+00
   1.35113e+00 -1.19094e+00 -8.55630e-01  1.62963e+00  1.69168e+00
   1.29602e+00  2.95910e-01 -7.26810e-01 -6.49590e-01 -1.79630e-01
  -7.99760e-01 -2.39820e-01 -7.67360e-01 -8.24640e-01  7.90610e-01
   6.76670e-01  1.07697e+00 -8.70300e-02 -1.09405e+00 -1.17889e+00
  -2.15550e-01 -1.13590e+00 -1.43743e+00 -5.09530e-01 -6.07720e-01
  -2.09750e-01 -1.50370e-01 -1.82500e-02 -5.22200e-02  1.91270e-01
   4.57120e-01  7.18500e-02  6.04260e-01  7.10940e-01  1.81100e-01
  -7.05900e-02 -5.64720e-01 -6.39080e-01 -3.46820e-01  4.00350e-01
   9.25730e-01  1.03454e+00  6.46660e-01  1.15000e-03  2.93000e-01
   9.36180e-01  7.14460e-01 -7.90610e-01 -7.51860e-01 -8.94430e-01
   1.23588e+00  1.38477e+00  1.13078e+00  1.32925e+00 -2.88870e-01
  -6.77820e-01  1.35055e+00  1.64411e+00 -8.55150e-01 -8.27040e-01
  -1.11348e+00]
 [-1.59845e+00 -1.44937e+00 -1.46663e+00 -1.68607e+00  1.58769e+00
   2.44410e-01 -1.2785

In [15]:
def J_softmax(h,y):
    a = 1e-10
    b = y*np.log(h+a)
    return b.sum() / 60000

In [16]:
def J_sigmond(h,y):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / 10000

In [22]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1.txt",delimiter=',')
w2 = np.loadtxt("init_w2.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.8
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[-0.02331557  0.75487222  0.31522608 -0.01142887  0.2074996 ]
 [-0.08008636 -0.41269909 -0.21522384  0.25654836  0.6871091 ]
 [ 0.76245267 -0.71914438 -0.08638688  0.54254934  0.00538371]
 [ 0.96756232  0.60750537  0.49552248  0.53806026 -0.59093955]
 [-0.75937566 -0.21188142  0.15543924 -0.72192877 -0.12820968]
 [ 0.88067534  0.41795787  0.83739517  0.39472659 -0.21944771]
 [ 0.23686246 -0.80266778 -0.14931057  0.09460963 -0.78729235]
 [ 0.31542117  0.7180761   0.48642186 -0.39563855 -0.36977477]
 [ 0.06304261 -0.65619493  0.26639526 -0.00784595  0.60699442]
 [ 0.21483727  0.49244527 -0.43063263  0.86585437 -0.17728309]]
[[ 0.30703589]
 [ 0.07762503]
 [ 0.53451211]
 [ 0.19440388]
 [ 0.97646424]
 [ 0.53598412]
 [ 0.13375454]
 [-0.44714695]
 [ 0.83131927]
 [ 0.04440001]]


In [106]:
"""
构建一个三层神经网络
一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段
取一般数据为训练样本数

数据归一化，分字段进行，然后 reshape(-1,1)
隐藏层 200 个神经元,输入到隐藏层，用reLu
输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy
w1 的 大小为 67*200
w2 的 大小为 201*1

"""

'\n构建一个三层神经网络\n一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段\n取一般数据为训练样本数\n\n数据归一化，分字段进行，然后 reshape(-1,1)\n隐藏层 200 个神经元,输入到隐藏层，用reLu\n输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy\nw1 的 大小为 67*200\nw2 的 大小为 201*1\n\n'

In [23]:

for i in range(2000):
    ############ 隐藏层输入 ############
    hid_input = np.matmul(x,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = reLu(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    hpfs_output = sigmond(hpfs_input).reshape(-1,1)
    print('hpfs_output shape is ',hpfs_output.shape,hpfs_output)
    print('y is ',y)
    ############ 完成正向传播 ############
    print('----------------完成正向传播 ----------------')
    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################
    
    d_cost_hyfs_input = hpfs_output - y
    print('对隐藏层输入求导')
    print(d_cost_hyfs_input.shape)
    print(d_cost_hyfs_input[0:10])
    print('隐藏层输出 ',hid_output.shape)
    print(hid_output.T)
    print(hid_output.dtype)
    print('d_cost_hyfs_input')
    print(d_cost_hyfs_input.shape)
    print(d_cost_hyfs_input[0:3])
    print(d_cost_hyfs_input.dtype)
    np.savetxt('hid_output.txt',hid_output,delimiter=',')
    np.savetxt('d_cost_hyfs_input.txt',d_cost_hyfs_input,delimiter=',')
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 10000
    print('d_cost_w2')
    print(d_cost_w2.shape)
    print(d_cost_w2[0:10])
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*d_reLu(hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 10000
    print('----------cost is -------------',J_sigmond(hpfs_output,y))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')
    if i==1 :
        break
    

hpfs_output shape is  (10000, 1) [[0.71029455]
 [0.99727136]
 [0.57614731]
 ...
 [0.89604038]
 [0.99736111]
 [0.66030311]]
y is  [[1]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]
----------------完成正向传播 ----------------
对隐藏层输入求导
(10000, 1)
[[-2.89705449e-01]
 [ 9.97271364e-01]
 [-4.23852691e-01]
 [ 8.41248488e-01]
 [ 9.99471364e-01]
 [ 5.79473585e-01]
 [-5.54073174e-05]
 [ 9.99785388e-01]
 [-7.00870002e-05]
 [ 9.99661695e-01]]
隐藏层输出  (10000, 101)
[[1.         1.         1.         ... 1.         1.         1.        ]
 [1.83025827 0.         1.64131489 ... 3.10692911 0.         0.25681511]
 [1.06644354 2.55055516 0.         ... 1.95765454 2.17357746 0.        ]
 ...
 [2.47091851 2.14005302 0.         ... 0.         1.32182477 2.57119403]
 [0.         0.         0.7888959  ... 0.         0.         2.20938617]
 [0.1583266  0.         0.70504493 ... 1.11047096 0.         0.67468147]]
float64
d_cost_hyfs_input
(10000, 1)
[[-0.28970545]
 [ 0.99727136]
 [-0.42385269]]
float64
d_cost_w2
(101, 1)
[[nan]
 [n

C:\Users\Seymour520\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less_equal
  
C:\Users\Seymour520\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


d_cost_w2
(101, 1)
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
(10000, 1) (101, 1)
----------cost is ------------- nan
w1 最大值位置 (array([], dtype=int64), array([], dtype=int64)) ,w1 最大值 nan
d_w1 最大值位置 (array([], dtype=int64), array([], dtype=int64)) ,d_w1 最大值 nan
